## Mnemonics

In [1]:
from mnemonic import Mnemonic
from cryptoHelpers import *

In [2]:
mnemo = Mnemonic("english")
words = mnemo.generate(128)
print(words)

sad forest jealous swap insane over intact deliver kiss know budget toddler


In [3]:
seed = mnemo.to_seed(words)
seed

b'\xd4\xb0\xba\xa5\x13\x83\xfa\xb8\xdf\xcc\x94\xff\xf569\x8e\x08\t\x8dL\xd2\xecX$v\xcdQv\x7fH~\x8ex\xee2>\xc1\xd1O5\x04}\xb7\xeak\x9c\xc6H\xdf\xb7S7\x8c\xb2\x83:\x82\x01(\xdd\xa3$\xe6p'

In [4]:
entropy = mnemo.to_entropy(words)
entropy

bytearray(b'\xbd\xabe\xdem\xa7O;\xdda\xd0z\xcf|vq')

## CRC16

Cyclic Redundancy Check

In [5]:
def crc16(data: bytes, poly: int = 0x8005, reg: int = 0x0000):
    # No reflections on input or output.
    # 
    # data: bytes - data to apply crc algorithm to
    # param: poly - generator polinomial, default 0b1000000000000101
    # actual generator is 0x18005, 0b11000000000000101 but msb gets checked before shift
    
    data = bytearray(data)
    
    # pad with zeros to width of poly, 16-bits
    data += bytearray(b'\x00\x00')

    for octet in data:
        for ii in range(8):
            msb = reg & 0x8000 # isolate most significant bit
            reg <<= 1 # shift the register
            reg ^= (octet >> 7-ii) & 0x01
            reg &= 0xFFFF # keep 16 bits, discard the rest
            if msb:
                reg ^= poly
            
            reg &= 0xFFFF # keep 16 bits, discard the rest
        
    return reg


In [6]:
data = bytearray('abcd','utf-8')
print(data)
print(hex(crc16(data)))

bytearray(b'abcd')
0x58e7


In [7]:
# alternative implementation, which I don't fully understand
# no padding of data and strange extra xor to msb if a bit in octet is a one
def crc16(data: bytes, poly: int = 0x8005, reg: int = 0x0000) -> int:
    # does not perform any reflections on input or output
    # param: data - bytes to apply checksum
    # param: poly - generator polinomial, default 0b1000000000000101
    if data is None:
        return 0

    for octet in data:
        
        for i in range(8):
            msb = reg & 0x8000
            if octet & (0x80 >> i):
                msb ^= 0x8000
            reg <<= 1
            if msb:
                reg ^= poly
        reg &= 0xFFFF
        
    return reg 

In [8]:
data = bytearray('abcd','utf-8')
print(data)
print(hex(crc16(data)))

bytearray(b'abcd')
0x58e7


## Generate Tezos Keys From Mnemonic

In [9]:
from mnemonic import Mnemonic
import hashlib as hl

In [10]:
# import the vocabulary
f = open('bip39english.txt', 'r')
vocab = [line.strip('\n') for line in f]
f.close()

In [11]:
# 160 gives 15 word mnemonic and 256 gives 24 words
mnemo = Mnemonic('english')
phrase = mnemo.generate(256)
print(phrase)
entropy = mnemo.to_entropy(phrase)
entropy_list = list(entropy)

champion escape slogan daughter section sand hero talent charge crew state pond retire service runway inquiry spice steel skirt turn bird organ change giraffe


In [12]:
entropy.hex()

'2609a32f9bec2b7e5ad6eb26866b5353eb7f886f5ba6d19aa72af5616938c98b'

In [13]:
eLength = len(entropy)*8 # length in bits
csLength = int(eLength/32) # checksum length
csLength

8

In [14]:
h = hl.sha256()
h.update(entropy)
entropy.append(h.digest()[0])

In [15]:
entropy.hex()

'2609a32f9bec2b7e5ad6eb26866b5353eb7f886f5ba6d19aa72af5616938c98b11'

In [16]:
# hash
mnemo.to_seed(phrase)

b'\x1d\xb1\xddF\xf9/8\x17`\xbf\xc5\xf2\xb7z\xa4K\x00\x15E\x8d"\xd1\x05\x8f]c\xa1\xc9\xbb\xfd(\xa7\xa9tZ\xb6\xe1\xabb>["\x1dX\xc1ri\xfbk\xda>a\xba\x0bK\xbfB\x98A\x89\x1cY\xf2z'

In [17]:
# convert byte array to bit array
bits = []
for byte in entropy:
    for i in range(8):
        if (byte & 0b10000000 >> i):
            bits.append(1)
        else:
            bits.append(0)
        
print(bits)

[0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1]


In [18]:
# iterate through with 11 bit chunks
my_words = []
for k in range(0,int(len(bits)/11)):
    chunk = bits[k*11:k*11+11]
    index = 0
    for j in range(11):
        index += chunk[j]*2**(11-1-j)
    my_words.append(vocab[index])
my_phrase = ' '.join(my_words)
print(my_phrase)

champion escape slogan daughter section sand hero talent charge crew state pond retire service runway inquiry spice steel skirt turn bird organ change giraffe


In [75]:
my_words

['champion',
 'escape',
 'slogan',
 'daughter',
 'section',
 'sand',
 'hero',
 'talent',
 'charge',
 'crew',
 'state',
 'pond',
 'retire',
 'service',
 'runway',
 'inquiry',
 'spice',
 'steel',
 'skirt',
 'turn',
 'bird',
 'organ',
 'change',
 'giraffe']

### Mnemonic to Seed

In [76]:
passphrase = ''
PBKDF2_ROUNDS = 2048
mnemonic = my_phrase
passphrase = "mnemonic" + passphrase
mnemonic_bytes = mnemonic.encode("utf-8")
passphrase_bytes = passphrase.encode("utf-8")
stretched = hl.pbkdf2_hmac("sha512", mnemonic_bytes, passphrase_bytes, PBKDF2_ROUNDS)
seed = stretched[:64]
seed

b'\x1d\xb1\xddF\xf9/8\x17`\xbf\xc5\xf2\xb7z\xa4K\x00\x15E\x8d"\xd1\x05\x8f]c\xa1\xc9\xbb\xfd(\xa7\xa9tZ\xb6\xe1\xabb>["\x1dX\xc1ri\xfbk\xda>a\xba\x0bK\xbfB\x98A\x89\x1cY\xf2z'

In [77]:
# for P256/secp256r1 secret exponent is the first 32 bytes of the seed
secret_exponent = seed[:32]
secret_exponent.hex()

'1db1dd46f92f381760bfc5f2b77aa44b0015458d22d1058f5d63a1c9bbfd28a7'

In [61]:
# P256
import fastecdsa.keys 
import fastecdsa.curve
from fastecdsa.encoding.util import bytes_to_int
import fastecdsa.encoding.sec1
from pyblake2 import blake2b
from pytezos.crypto.encoding import base58_decode, base58_encode

pk = fastecdsa.keys.get_public_key(bytes_to_int(secret_exponent), curve=fastecdsa.curve.P256)
public_point = fastecdsa.encoding.sec1.SEC1Encoder.encode_public_key(pk)

In [63]:
pk

X: 0xd74c9f1235de2f3fa48c6690cb96472a374ecca57a14611f510d1efcb0bd6b80
Y: 0x9bb0edbd824de9b8838f2f91005d1c4b922bf32065748ebe1937879dd59c1096
(On curve <P256>)

In [64]:
public_point

b'\x02\xd7L\x9f\x125\xde/?\xa4\x8cf\x90\xcb\x96G*7N\xcc\xa5z\x14a\x1fQ\r\x1e\xfc\xb0\xbdk\x80'

In [65]:
public_point.hex()

'02d74c9f1235de2f3fa48c6690cb96472a374ecca57a14611f510d1efcb0bd6b80'

In [42]:
len(public_point)

33

In [43]:
pkh = blake2b(data=public_point, digest_size=20).digest()
print(pkh.hex())
prefix = b'tz3'
public_hash = base58_encode(pkh, prefix).decode()

a64ddd59755e9aa9170290fa6c661a5e822b813a


In [44]:
public_hash

'tz3bVP4dsGWhTpBjkuef7Pww8fSEFSQddfZW'

In [50]:
len(public_point)

33

In [101]:
# try a different public point
pp = bytes([0x02, 0xc5, 0x82, 0x56, 0xe7, 0xe8, 0x3c, 0xf0, 0xd0, 0x17, 0x6b,
            0x25, 0x56, 0x08, 0xd7, 0x41, 0x79, 0x85, 0xdf, 0x69, 0x51, 0x9c,
            0x2B, 0x14, 0xd1, 0x29, 0xe8, 0xd5, 0x6a, 0x3e, 0xb5, 0x38, 0xde])


pkh = blake2b(data=pp, digest_size=20).digest()
print(pkh.hex())
prefix = b'tz3'
public_hash = base58_encode(pkh, prefix).decode()
print(public_hash)

98fe664776a112c744236d75f93625d3ea70356e
tz3aH15ouCQdPLpizWnqKNidAxNMXUfKNKFv


In [96]:
prefix = b'\x03\xb2\x8b\x7f'
prefix.hex()

'03b28b7f'

In [102]:
def tb(l):
    return b''.join(map(lambda x: x.to_bytes(1, 'big'), l))

In [103]:
tb([6, 161, 164]).hex()

'06a1a4'

## Compare to PyTezos Results

In [45]:
import pytezos as pytezos
from pytezos.crypto.key import Key

In [69]:
p2pk = Key.from_public_point(
    bytes.fromhex('02d74c9f1235de2f3fa48c6690cb96472a374ecca57a14611f510d1efcb0bd6b80'), 
    curve=b'p2')
p2pk


Public key hash
tz3bVP4dsGWhTpBjkuef7Pww8fSEFSQddfZW

Helpers
.blinded_public_key_hash()
.from_alias()
.from_encoded_key()
.from_faucet()
.from_mnemonic()
.from_public_point()
.from_secret_exponent()
.generate()
.public_key()
.public_key_hash()
.secret_key()
.sign()
.verify()

In [70]:
sk = Key.from_mnemonic(mnemonic = my_words, curve=b'p2')

In [71]:
sk.public_key_hash()

'tz3bVP4dsGWhTpBjkuef7Pww8fSEFSQddfZW'

In [72]:
message = sk.public_key()
message

'p2pk66CyvDM2vfHmGbHHHWZGgYqL5VtxUFaMYFpFaMZTcQAZ1iriQLs'

In [56]:
len(sk.public_key())

55

In [73]:
sk.secret_key()

'p2sk2ZsKUyRG8yJ3QcNNwjdx3qUCqpx5Myeb6QhRMeSdRJBYiqzupz'

In [58]:
secret_exponent.hex()

'1db1dd46f92f381760bfc5f2b77aa44b0015458d22d1058f5d63a1c9bbfd28a7'

In [81]:
p2pkb = Key.from_secret_exponent(secret_exponent, curve=b'p2')

In [87]:
p2skb = p2pkb.secret_key()
print(p2skb)

p2sk2ZsKUyRG8yJ3QcNNwjdx3qUCqpx5Myeb6QhRMeSdRJBYiqzupz


In [95]:
b'\016\081\238\189'.hex()

'0e0038311338013839'

### Base58 Encoding

In [152]:
b58_alphabet = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'

In [153]:
message = 'Hello World!'
message = bytearray(message, 'utf-8')

In [154]:
for each in message:
    print(each)

72
101
108
108
111
32
87
111
114
108
100
33


In [155]:
# try to adapt some c++ code to do base58 encoding

data = bytearray(b'Hello World!')
# prepend = bytearray(b'\x00\x00')
prepend = bytearray(b'')
data = prepend + data

base58map = ['1', '2', '3', '4', '5', '6', '7', '8',
             '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G',
             'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q',
             'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y',
             'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g',
             'h', 'i', 'j', 'k', 'm', 'n', 'o', 'p',
             'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
             'y', 'z' ]
b58_size = int(len(data)*138/100) + 1 # minimum size of b58 encode
digits = [0]*b58_size
digitslen = 1

encode_flag = False
leading_zeros = 0

for i in range(len(data)):
    if (not(encode_flag) and data[i] == 0):
        leading_zeros = leading_zeros + 1
    if (not(encode_flag) and data[i] != 0):
        encode_flag = True
   
    if (encode_flag):
        carry = data[i] # carry needs to be uint32_t in C++
        for j in range(digitslen):
            carry = carry + (digits[j]<<8) # digits[j] must be recast as a uint32_t in C++, same as <<8
            digits[j] = carry%58
            carry = int(carry/58)
        while (carry > 0):
            digits[digitslen] = carry%58
            digitslen = digitslen+1
            carry=int(carry/58)

# trim unused digits from digits
digits = digits[:digitslen]
            
for k in range(leading_zeros):
    digits.append(0)

digits.reverse()

base58_data = []
for each in digits:
    base58_data.append(base58map[each])
print(base58_data)

['2', 'N', 'E', 'p', 'o', '7', 'T', 'Z', 'R', 'R', 'r', 'L', 'Z', 'S', 'i', '2', 'U']


### Blake2b Hash

Implemented with a hash length of 20 bytes and no key utilization.

Blake2b does mathamatical operations on 64-bit words but here these are recast as operations on 8, 8-bit bytes because I want to implement eventually on an microcontroller.

In [156]:
# parameters
w = 64 # bits in words
r = 12 # rounds of message mixing
bb = 128 # bytes in each block/chunk
hash_bytes = 20 # bytes in hash, 1 to 64
# rotations
R1 = 32
R2 = 24
R3 = 16
R4 = 63

# initialization vector
IV0 = 0x6a09e667f3bcc908   # Frac(sqrt(2))
IV1 = 0xbb67ae8584caa73b   # Frac(sqrt(3))
IV2 = 0x3c6ef372fe94f82b   # Frac(sqrt(5))
IV3 = 0xa54ff53a5f1d36f1   # Frac(sqrt(7))
IV4 = 0x510e527fade682d1   # Frac(sqrt(11))
IV5 = 0x9b05688c2b3e6c1f   # Frac(sqrt(13))
IV6 = 0x1f83d9abfb41bd6b   # Frac(sqrt(17))
IV7 = 0x5be0cd19137e2179   # Frac(sqrt(19))

IV = [[0x6a, 0x09, 0xe6, 0x67, 0xf3, 0xbc, 0xc9, 0x08],
      [0xbb, 0x67, 0xae, 0x85, 0x84, 0xca, 0xa7, 0x3b],
      [0x3c, 0x6e, 0xf3, 0x72, 0xfe, 0x94, 0xf8, 0x2b],
      [0xa5, 0x4f, 0xf5, 0x3a, 0x5f, 0x1d, 0x36, 0xf1],
      [0x51, 0x0e, 0x52, 0x7f, 0xad, 0xe6, 0x82, 0xd1],
      [0x9b, 0x05, 0x68, 0x8c, 0x2b, 0x3e, 0x6c, 0x1f],
      [0x1f, 0x83, 0xd9, 0xab, 0xfb, 0x41, 0xbd, 0x6b],
      [0x5b, 0xe0, 0xcd, 0x19, 0x13, 0x7e, 0x21, 0x79]]

# message word permutation schedule
SIGMA0 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
SIGMA1 = [14, 10, 4, 8, 9, 15, 13, 6, 1, 12, 0, 2, 11, 7, 5, 3]
SIGMA2 = [11, 8, 12, 0, 5, 2, 15, 13, 10, 14, 3, 6, 7, 1, 9, 4]
SIGMA3 = [7, 9, 3, 1, 13, 12, 11, 14, 2, 6, 5, 10, 4, 0, 15, 8]
SIGMA4 = [9, 0, 5, 7, 2, 4, 10, 15, 14, 1, 11, 12, 6, 8, 3, 13]
SIGMA5 = [2, 12, 6, 10, 0, 11, 8, 3, 4, 13, 7, 5, 15, 14, 1, 9]
SIGMA6 = [12, 5, 1, 15, 14, 13, 4, 10, 0, 7, 6, 3, 9, 2, 8, 11]
SIGMA7 = [13, 11, 7, 14, 12, 1, 3, 9, 5, 0, 15, 4, 8, 6, 2, 10]
SIGMA8 = [6, 15, 14, 9, 11, 3, 0, 8, 12, 2, 13, 7, 1, 4, 10, 5]
SIGMA9 = [10, 2, 8, 4, 7, 6, 1, 5, 15, 11, 9, 14, 3, 12, 13, 0]

SIGMA = [SIGMA0, SIGMA1, SIGMA2, SIGMA3, SIGMA4, SIGMA5, SIGMA6, SIGMA7, SIGMA8, SIGMA9]

In [110]:
# blak2b mix function
def blake2b_mix(Va, Vb, Vc, Vd, x, y):
    # Inputs:
    # Va, Vb, Vc, Vd       four 8-byte word entries from the work vector V
    # x, y                two 8-byte word entries from padded message m
    Va = (Va + Vb + x) % (2**w)
    Vd = rotate_bits_right(Vd ^ Va, R1)
    Vc = (Vc + Vd) % (2**w)
    Vb = rotate_bits_right(Vb ^ Vc, R2)
    Va = (Va + Vb + y) % (2**w)
    Vd = rotate_bits_right(Vd ^ Va, R3)
    Vc = (Vc + Vd) % (2**w)
    Vb = rotate_bits_right(Vb ^ Vc, 63)
    
    # Output
    return Va, Vb, Vc, Vd

def rotate_list_right(V, R):
    # rotate V right by R bits
    return V[-R:]+V[:-R]

def rotate_bits_right(V, R, w):
    return V>>R ^ (V<<(w-R)) % 2**w

def rotate_bytes_right(V, R, w):
    # V is an array of bytes interpreted as a stream in big endian fashion
    # w is 8 for a byte array
    n = len(V) # number of bytes in array
    RF = int(R/w) # number of full byte shifts
    RP = R%w # number of partial byte shifts
    V = rotate_list_right(V,RF)
    V = [(V[0]>>RP) ^ (V[n-1]<<(w-RP))%2**w] + [(V[i]>>RP) ^ (V[i-1]<<(w-RP))%2**w for i in range(1,n)]
    return V

def byte_stream_to_word(P):
    # convert a byte stream (bytearray eight bytes long) in little endian fashion 
    # into a word (also eight bytes long)
    return bytearray([P[7-i] for i in range(8)])

In [111]:
V = bytearray(b'abc')
for each in V:
    print(bin(each))

0b1100001
0b1100010
0b1100011


In [112]:
VX = rotate_bytes_right(V,24,8)

In [113]:
for each in VX:
    print(bin(each))

0b1100001
0b1100010
0b1100011


In [157]:
pkh = blake2b(data=b'abc', digest_size=20).digest()
print(pkh.hex())

384264f676f39536840523f284921cdc68b6846b


In [115]:
V = bytearray(b'abcdefgh')
byte_stream_to_word(V)

bytearray(b'hgfedcba')

In [ ]:
b = [0]*128 # input buffer, 128 bytes (unit8_t)
h = [0]*8 # chained state, 8 64-bit words (uint64_t)
t = [0]*2 # total number of bytes, 2 64-bit values, high bit, low bit (uint64_t)
# size_t c # pointer for b[]
outlen = 20 # digest size (size_t)

